# Extracting Gmail Data

This process consists of 3 steps:

1. Enable Gmail API (Google Cloud Console)
2. Authenticate Python to access Gmail API 
3. Start Extraction

## Authenticating Python

In [ ]:
# Installing necessary libraries
!pip install --upgrade google-api-python-client google-auth google-auth-oauthlib google-auth-httplib2 pandas


In [2]:
# Importing necessary modules
import os
import pickle
import pandas as pd

from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request

# Error due to libraries being installed on a different kernel than the one Jupyter is using

ModuleNotFoundError: No module named 'googleapiclient'

In [ ]:
# Identifying the required kernel being used by Jupyter
import sys
print(sys.executable)


In [ ]:
# Tried installing one library first
!"C:\Users\u411296\AppData\Local\anaconda3\python.exe" -m pip install google-auth-oauthlib


In [1]:
# Tried importing the module from that library
from google_auth_oauthlib.flow import InstalledAppFlow
print("✅ google_auth_oauthlib imported")

# Success

✅ google_auth_oauthlib imported


In [3]:
# Installing and importing the rest of the library and modules the same way
!"C:\Users\u411296\AppData\Local\anaconda3\python.exe" -m pip install google-api-python-client google-auth google-auth-httplib2


  Obtaining dependency information for google-api-python-client from https://files.pythonhosted.org/packages/96/58/c1e716be1b055b504d80db2c8413f6c6a890a6ae218a65f178b63bc30356/google_api_python_client-2.187.0-py3-none-any.whl.metadata
  Using cached google_api_python_client-2.187.0-py3-none-any.whl.metadata (7.0 kB)
  Obtaining dependency information for google-auth-httplib2 from https://files.pythonhosted.org/packages/44/a7/ca23dd006255f70e2bc469d3f9f0c82ea455335bfd682ad4d677adc435de/google_auth_httplib2-0.2.1-py3-none-any.whl.metadata
  Using cached google_auth_httplib2-0.2.1-py3-none-any.whl.metadata (3.0 kB)
  Obtaining dependency information for httplib2<1.0.0,>=0.19.0 from https://files.pythonhosted.org/packages/8c/a2/0d269db0f6163be503775dc8b6a6fa15820cc9fdc866f6ba608d86b721f2/httplib2-0.31.0-py3-none-any.whl.metadata
  Using cached httplib2-0.31.0-py3-none-any.whl.metadata (2.2 kB)
  Obtaining dependency information for google-api-core!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.0,<3.0.0,>=1

In [2]:
import os
import pickle
import pandas as pd

from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request


In [5]:
os.getcwd() # The CWD is one step inside the root

'C:\\Users\\u411296\\OneDrive - United Airlines\\Documents\\My Space\\Upskilling Msyelf\\Machine Learning\\Personal Gmail Classifier ML Project\\notebooks'

In [6]:
os.chdir('..') # Changing the CWD to one folder above (root) since the credentials file is located there

In [7]:
# Authorizing Python to access Gmail

## --> Access Denied initially
## --> Added my email as a test user in the project created in Goggle Cloud Console

from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
import os
import pickle

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

creds = None

# Load existing token
if os.path.exists('token.json'):
    with open('token.json', 'rb') as token:
        creds = pickle.load(token)

# If no valid credentials, login
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
    'credentials.json', SCOPES
)
        creds = flow.run_local_server(port=0)

    # Save token
    with open('token.json', 'wb') as token:
        pickle.dump(creds, token)

# Build Gmail service
service = build('gmail', 'v1', credentials=creds)

print("✅ Gmail API authenticated successfully")


# Success

✅ Gmail API authenticated successfully


## Testing Extraction

In [14]:
# Tried fetching 10 mails
results = service.users().messages().list(
    userId='me',
    maxResults=10
).execute()

messages = results.get('messages', [])

print(f"Fetched {len(messages)} message IDs")


Fetched 10 message IDs


In [15]:
def get_header(headers, name):
    for h in headers:
        if h['name'].lower() == name.lower():
            return h['value']
    return ""

for i, msg in enumerate(messages, start=1):
    message = service.users().messages().get(
        userId='me',
        id=msg['id'],
        format='metadata',
        metadataHeaders=['From', 'Subject']
    ).execute()

    headers = message['payload']['headers']
    
    subject = get_header(headers, 'Subject')
    sender = get_header(headers, 'From')
    snippet = message.get('snippet', '')

    print(f"\n📧 Email {i}")
    print("From   :", sender)
    print("Subject:", subject)
    print("Snippet:", snippet[:200])  # limit print



📧 Email 1
From   : Scapia <no-reply@offers.scapia.cards>
Subject: An Apple Watch under your tree 🎄
Snippet: A few friends. One festive surprise waiting for you. ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏

📧 Email 2
From   : ICICI Bank <services@custcomm.icicibank.com>
Subject: Get exciting rewards using ICICI Bank Visa Cards!
Snippet: Travel More. Earn More. Get exclusive offers on flight bookings Dear Customer, • Get flat ₹250 off on domestic flights • Get flat ₹600 off on international flights Coupon Code: VISAFLY Book via the Ai

📧 Email 3
From   : "Kotak Fastrack Personal Loan 💰" <retailproducts@mail.kotak.in>
Subject: Hey Priya, Need ₹1,00,000 fast? You’re already approved! ✅
Snippet: Tap into instant Personal Loan‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌

## Extracting emails

> Extracting only the **high-signal, low-noise** fields from the mail

1. message_id
2. subject
3. snippet
4. sender
5. sender_domain
6. internal_date
7. labels
8. has_attachment

In [9]:
import os
import pandas as pd

# Helper to extract headers safely
def get_header(headers, name):
    for h in headers:
        if h['name'].lower() == name.lower():
            return h['value']
    return ""

emails = []
next_page_token = None
total_fetched = 0

while True:
    response = service.users().messages().list(
        userId='me',
        maxResults=500,
        pageToken=next_page_token
    ).execute()

    messages = response.get('messages', [])

    for msg in messages:
        message = service.users().messages().get(
            userId='me',
            id=msg['id'],
            format='metadata',
            metadataHeaders=['From', 'Subject']
        ).execute()

        headers = message['payload']['headers']

        sender = get_header(headers, 'From')
        subject = get_header(headers, 'Subject')

        # Sender domain extraction
        sender_domain = ""
        if "@" in sender:
            sender_domain = sender.split("@")[-1].replace(">", "").strip()

        # Attachment signal
        has_attachment = False
        payload = message.get("payload", {})
        if "parts" in payload:
            for part in payload["parts"]:
                if part.get("filename"):
                    has_attachment = True
                    break

        emails.append({
            "message_id": msg["id"],
            "subject": subject,
            "snippet": message.get("snippet", ""),
            "sender": sender,
            "sender_domain": sender_domain,
            "internal_date": message.get("internalDate"),
            "labels": message.get("labelIds", []),
            "has_attachment": has_attachment
        })

        total_fetched += 1

    next_page_token = response.get("nextPageToken")
    if not next_page_token:
        break

print(f"✅ Extracted {total_fetched} emails")


✅ Extracted 69672 emails


## Converting the list of dictionaries (emails) into a Pandas Dataframe

In [10]:
df = pd.DataFrame(emails)

df.head()


,message_id,subject,snippet,sender,sender_domain,internal_date,labels,has_attachment
0,19b27b8215ee9601,Give the Gift of Smart Tech | ESR,Get an extra 20% off sitewide with code 25XMAS...,ESRTech <newsletter@esrtech.com>,esrtech.com,1765897288000,"[CATEGORY_PROMOTIONS, UNREAD, INBOX]",False
1,19b271387099e3e7,The strength of working women in Afghanistan,Plus: A tumultuous history of punctuation ͏ ͏ ...,Aeon Daily <support@aeon.co>,aeon.co,1765886455000,"[UNREAD, CATEGORY_UPDATES, INBOX]",False
2,19b269adbb77da49,"Dear Customer, Important Update for K7 Antivir...",Don&#39;t Wait Till It&#39;s Too Late HACKERS ...,K7 Antivirus <alerts@k7computing.com>,k7computing.com,1765877280000,"[UNREAD, CATEGORY_UPDATES, INBOX]",False
3,19b267b7d35b4aa3,Last payment attempt unsuccessful for Jio Numb...,"Dear Customer, Your last payment attempt of Rs...",Notification@jio.com,jio.com,1765877183000,"[UNREAD, CATEGORY_UPDATES, INBOX]",False
4,19b266e80aebd978,Last payment attempt unsuccessful for Jio Numb...,"Dear Customer, Your last payment attempt of Rs...",Notification@jio.com,jio.com,1765876333000,"[UNREAD, CATEGORY_UPDATES, INBOX]",False


## Loading the Pandas Dataframe into a CSV File

In [12]:
os.getcwd() # The CWD is one step inside the root

'C:\\Users\\u411296\\OneDrive - United Airlines\\Documents\\My Space\\Upskilling Msyelf\\Machine Learning\\Personal Gmail Classifier ML Project'

In [15]:
df.to_csv(r"C:\Users\u411296\OneDrive - United Airlines\Documents\My Space\Upskilling Msyelf\Machine Learning\Personal Gmail Classifier ML Project\data\raw_emails.csv", index=False)
print("Data loaded into CSV successfully")

Data loaded into CSV successfully


In [16]:
df = pd.DataFrame(emails)

df.head()


,message_id,subject,snippet,sender,sender_domain,internal_date,labels,has_attachment
0,19b27b8215ee9601,Give the Gift of Smart Tech | ESR,Get an extra 20% off sitewide with code 25XMAS...,ESRTech <newsletter@esrtech.com>,esrtech.com,1765897288000,"[CATEGORY_PROMOTIONS, UNREAD, INBOX]",False
1,19b271387099e3e7,The strength of working women in Afghanistan,Plus: A tumultuous history of punctuation ͏ ͏ ...,Aeon Daily <support@aeon.co>,aeon.co,1765886455000,"[UNREAD, CATEGORY_UPDATES, INBOX]",False
2,19b269adbb77da49,"Dear Customer, Important Update for K7 Antivir...",Don&#39;t Wait Till It&#39;s Too Late HACKERS ...,K7 Antivirus <alerts@k7computing.com>,k7computing.com,1765877280000,"[UNREAD, CATEGORY_UPDATES, INBOX]",False
3,19b267b7d35b4aa3,Last payment attempt unsuccessful for Jio Numb...,"Dear Customer, Your last payment attempt of Rs...",Notification@jio.com,jio.com,1765877183000,"[UNREAD, CATEGORY_UPDATES, INBOX]",False
4,19b266e80aebd978,Last payment attempt unsuccessful for Jio Numb...,"Dear Customer, Your last payment attempt of Rs...",Notification@jio.com,jio.com,1765876333000,"[UNREAD, CATEGORY_UPDATES, INBOX]",False
